Install exifread

In [ ]:
!pip install exifread

Import exifread along with a few other modules to handle inspecting this data.

In [ ]:
import os
import glob
import pprint
import exifread

We'll need some sample data to process EXIF tags on. You may have a batch of photos but for consistency let's use the material from here so we can expect the same results.

https://github.com/ianare/exif-samples

https://github.com/ianare/exif-samples/archive/master.zip

The below assumes you've extracted this zip file under the current working directory.

In [ ]:
cwd = os.path.abspath('.')

if not os.path.exists(os.path.join(cwd, 'exif-samples-master')):
    print("exif-samples-master not found")

With the files in place, we can inspect the dictionary dump of the results. This produces a lot of results so pprint will come in handed here.

In [ ]:
for filename in glob.glob('petra/*.jpg', recursive=True):
    
    # Open image file for reading (binary mode)
    fh = open(os.path.join(cwd, filename), 'rb')

    # Return Exif tags
    tags = exifread.process_file(fh)
    
    pprint.pprint(tags)

With that, we can define some functionality to pull out and parse out the GPS coordinates that get embedded in EXIF data. We don't have to reinvent the wheel here and can use material that is out there publicly. This is from https://gist.github.com/snakeye/fdc372dbf11370fe29eb.

In [ ]:
# https://gist.github.com/snakeye/fdc372dbf11370fe29eb

def _get_if_exist(data, key):
    if key in data:
        return data[key]

    return None


def _convert_to_degress(value):
    """
    Helper function to convert the GPS coordinates stored in the EXIF to degress in float format
    :param value:
    :type value: exifread.utils.Ratio
    :rtype: float
    """
    d = float(value.values[0].num) / float(value.values[0].den)
    m = float(value.values[1].num) / float(value.values[1].den)
    s = float(value.values[2].num) / float(value.values[2].den)

    return d + (m / 60.0) + (s / 3600.0)
    
def get_exif_location(exif_data):
    """
    Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)
    """
    lat = None
    lon = None

    gps_latitude = _get_if_exist(exif_data, 'GPS GPSLatitude')
    gps_latitude_ref = _get_if_exist(exif_data, 'GPS GPSLatitudeRef')
    gps_longitude = _get_if_exist(exif_data, 'GPS GPSLongitude')
    gps_longitude_ref = _get_if_exist(exif_data, 'GPS GPSLongitudeRef')

    if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
        lat = _convert_to_degress(gps_latitude)
        if gps_latitude_ref.values[0] != 'N':
            lat = 0 - lat

        lon = _convert_to_degress(gps_longitude)
        if gps_longitude_ref.values[0] != 'E':
            lon = 0 - lon

    return lat, lon

With the above defined, let's try some functionality to inspect our results.

In [ ]:
for filename in glob.glob('exif-samples-master/**/*.jpg', recursive=True):
    # Open image file for reading (binary mode)
    fh = open(filename, 'rb')

    try:
        # Return Exif tags
        tags = exifread.process_file(fh)
    
        gps_tuple = get_exif_location(tags)
    
        if gps_tuple != (None, None):
            print(filename, gps_tuple)

    # there are known corrupt files, we must handle this
    except OSError:
        pass
    except ZeroDivisionError:
        pass

With the above worked out, let's try to visualize these results more effectively.

In [ ]:
!pip install folium

In [ ]:
import folium

Let's center on the Eiffel Tower in Paris since we have multiple images with Europe EXIF data

In [ ]:
map_ = folium.Map(location=[48.8584, 2.2945], zoom_start=3)

Try it out! Swap the print of lat/lon for marking up a map.

In [ ]:
for filename in glob.glob('exif-samples-master/**/*.jpg', recursive=True):
    # Open image file for reading (binary mode)
    fh = open(filename, 'rb')

    try:
        # Return Exif tags
        tags = exifread.process_file(fh)
    
        gps_tuple = get_exif_location(tags)
    
        if gps_tuple != (None, None):
            # print(filename, gps_tuple)
            folium.Marker(gps_tuple).add_to(map_)

    # there are known corrupt files, we must handle this
    except OSError:
        pass
    except ZeroDivisionError:
        pass

Display it!

In [ ]:
map_